Food environment dataset - where are good counties to start a food business.


Where is best to start a new restaurant business in the US?

In [19]:
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cartopy'

In [74]:
# Read the relevant sheets of the Excel file as a dictionary of Pandas dateframes
dict_FEA = pd.read_excel('Food_Environment_Atlas.xls', ['Supplemental Data - County', 'RESTAURANTS'])

# Extract population and restaurant data as separate dataframes
df_FEA_pop = dict_FEA['Supplemental Data - County']
df_FEA_rest = dict_FEA['RESTAURANTS']

Inspect column headings to understand data structure

In [75]:
print(df_FEA_pop.columns)
print(df_FEA_rest.columns)

Index(['FIPS ', 'State', 'County', '2010 Census Population',
       'Population Estimate, 2011', 'Population Estimate, 2012',
       'Population Estimate, 2013', 'Population Estimate, 2014',
       'Population Estimate, 2015', 'Population Estimate, 2016'],
      dtype='object')
Index(['FIPS', 'State', 'County', 'FFR09', 'FFR14', 'PCH_FFR_09_14',
       'FFRPTH09', 'FFRPTH14', 'PCH_FFRPTH_09_14', 'FSR09', 'FSR14',
       'PCH_FSR_09_14', 'FSRPTH09', 'FSRPTH14', 'PCH_FSRPTH_09_14',
       'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12'],
      dtype='object')


The first heading of the population dataframe has a space after 'FIPS'. Let's correct that by renaming the column.

In [76]:
df_FEA_pop.rename(columns = {'FIPS ':'FIPS'}, inplace = True)

I want to combine the population and the restaurant dataframes, but I first need to make sure that their shapes are compatible.

In [78]:
# Shape of the dataframe of the county population data
print(df_FEA_pop.shape)

# Shape of the dataframe of the restaurant data
print(df_FEA_rest.shape)

(3142, 10)
(3143, 19)


There is one more row in the restaurants data compared to the population data so the dataframes cannot yet be combined. The rows correspond to US counties. After a quick wikipedia search, there are actually 3142 counties in the US.

One possibility is that there are 

In [79]:
dict_FEA['RESTAURANTS']['FIPS'].duplicated()

#dict_FEA['Supplemental Data - County']['FIPS '].duplicated().sum()

df1 = df_FEA_pop['FIPS']
df2 = df_FEA_rest['FIPS']

print(df1[~df1.isin(df2)])
print(df2[~df2.isin(df1)])

81       2158
2412    46102
Name: FIPS, dtype: int64
92       2270
2417    46113
2916    51515
Name: FIPS, dtype: int64


In [81]:
print(df_FEA_pop.iloc[[80, 81, 82, 2411, 2412, 2413]][['FIPS', 'State', 'County']])
print(df_FEA_rest.iloc[[80, 81, 82, 2411, 2412, 2413]][['FIPS', 'State', 'County']])

       FIPS          State               County
80     2150         Alaska       Kodiak Island 
81     2158         Alaska            Kusilvak 
82     2164         Alaska  Lake and Peninsula 
2411  46101   South Dakota               Moody 
2412  46102   South Dakota       Oglala Lakota 
2413  46103   South Dakota          Pennington 
       FIPS State              County
80     2150    AK       Kodiak Island
81     2164    AK  Lake and Peninsula
82     2170    AK   Matanuska-Susitna
2411  46101    SD               Moody
2412  46103    SD          Pennington
2413  46105    SD             Perkins


In [83]:
print(df_FEA_pop.iloc[[91, 92, 93, 2416, 2417, 2418, 2915, 2916, 2917]][['FIPS', 'State', 'County']])
print(df_FEA_rest.iloc[[91, 92, 93, 2416, 2417, 2418, 2915, 2916, 2917]][['FIPS', 'State', 'County']])

       FIPS          State                County
91     2240         Alaska  Southeast Fairbanks 
92     2261         Alaska       Valdez-Cordova 
93     2275         Alaska             Wrangell 
2416  46109   South Dakota              Roberts 
2417  46111   South Dakota              Sanborn 
2418  46115   South Dakota                Spink 
2915  51510       Virginia       Alexandria city
2916  51520       Virginia          Bristol city
2917  51530       Virginia      Buena Vista city
       FIPS State          County
91     2261    AK  Valdez-Cordova
92     2270    AK    Wade Hampton
93     2275    AK        Wrangell
2416  46111    SD         Sanborn
2417  46113    SD         Shannon
2418  46115    SD           Spink
2915  51510    VA      Alexandria
2916  51515    VA         Bedford
2917  51520    VA         Bristol


In [99]:
# Define new dataframes as subsets of population and restaurant dataframes but where rows which
# are not contained in both are removed. Reindex so that indices are compatible.
df_FEA_pop2 = df_FEA_pop[df_FEA_pop['FIPS'].isin(df_FEA_rest['FIPS'])].reset_index(drop=True)
df_FEA_rest2 = df_FEA_rest[df_FEA_rest['FIPS'].isin(df_FEA_pop['FIPS'])].reset_index(drop=True)

# Check that the number of rows match now.
print(df_FEA_pop2.shape, df_FEA_rest2.shape)

# Final check that the county identification numbers (FIPS) now match up (should be 0 if all match).
print((df_FEA_pop2['FIPS'] != df_FEA_rest2['FIPS']).sum())

(3140, 10) (3140, 19)
0


Now we can combine the dataframes into one.

In [106]:
# Combine dataframes into one, omitting the FIPS, state, and county columns as they are already in the
# population dateframe.
df_FEA = pd.concat([df_FEA_pop2, df_FEA_rest2.drop(['FIPS', 'State', 'County'], axis=1)], axis=1)

Index(['FIPS', 'State', 'County', '2010 Census Population',
       'Population Estimate, 2011', 'Population Estimate, 2012',
       'Population Estimate, 2013', 'Population Estimate, 2014',
       'Population Estimate, 2015', 'Population Estimate, 2016', 'FFR09',
       'FFR14', 'PCH_FFR_09_14', 'FFRPTH09', 'FFRPTH14', 'PCH_FFRPTH_09_14',
       'FSR09', 'FSR14', 'PCH_FSR_09_14', 'FSRPTH09', 'FSRPTH14',
       'PCH_FSRPTH_09_14', 'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07',
       'PC_FSRSALES12'],
      dtype='object')


In [ ]:
# Plot map

fig = plt.figure(figsize=[19,10])

# choose map projection
ax = plt.axes(projection=ccrs.Robinson())

# Add map features
ax.coastlines()
ax.add_feature(ct.feature.OCEAN)
ax.add_feature(ct.feature.BORDERS)
#ax.add_feature(ct.feature.LAND, edgecolor='black')
#ax.add_feature(ct.feature.LAKES, edgecolor='black')
#ax.add_feature(ct.feature.RIVERS)

date = col_names[-1] # -1 for latest, 4 for earliest

# size of points
abs_size = 50000
def size_func(data):
    return abs_size * data/con_df_ungrouped[col_names[-1]].max()

sizes = size_func(con_df_ungrouped[date])
scat = plt.scatter(con_df_ungrouped['Long'], con_df_ungrouped['Lat'],
         color='r', s=sizes, marker='o', alpha=0.5,
         transform=ccrs.Geodetic())
scale_size = abs_size*10000/con_df_ungrouped[col_names[-1]].max()
plt.scatter([-150],[-30], color='r', s=scale_size, transform=ccrs.Geodetic())
plt.text(-150,-30, '10,000 people', transform=ccrs.Geodetic(), ha='center',
         fontsize=20)
date_text = plt.text(-150,0, 'Date: ' + date, transform=ccrs.Geodetic(),
                     ha='center', fontsize=20)
plt.title('Global Covid-19 cases', fontsize=30)
ax.set_global() # shows the whole map

In [57]:
# Download counties at https://www.usgs.gov/core-science-systems/national-geospatial-program/small-scale-data?openChapters=chpbound#chpbound

reader = shpreader.Reader('countyl010g.shp')

counties = list(reader.geometries())

COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND.with_scale('50m'))
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAKES.with_scale('50m'))
ax.add_feature(COUNTIES, facecolor='none', edgecolor='gray')

ax.coastlines('50m')

ax.set_extent([-83, -65, 33, 44])
plt.show()

NameError: name 'shpreader' is not defined